In [1]:
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import PyPDF2

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(file):
    
    rsrcmgr = PDFResourceManager()
    
    retstr = StringIO()
    
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams = laparams)
    fp = open(file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching=caching, check_extractable = True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
from urllib import request
from PyPDF2 import pdf
from urllib.request import urlopen

In [5]:
import os
import pandas as pd
# result_list = []
for i in range(1, 45):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.content)
    li_list = []
    li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
#     print(len(li_list))
#     print(li_list)
    print(i)
    result_list = []
    for x in range(0, len(li_list)):
        new_dict = {}
        link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
        for link in link_li:
            if link.find('a').attrs['title'][-3:] == 'pdf':
                link_u = link.find('a').attrs['href']
                title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                url2 = 'http://www.bok.or.kr' + link_u
                file_res2 = requests.get(url2)
                file_name = '{}.pdf'.format(title)
                with open(file_name, 'wb') as f:
                    f.write(file_res2.content)
                try:
                    text = convert_pdf_to_txt(file_name)
                    new_dict = {
                        'title' : title,
                        'text' : text
                    }
                    result_list.append(new_dict)

                except:
                    print('빠진 목록 : {}'.format(title))
                    
                              
    df = pd.DataFrame(result_list)
    BASE_DIR = os.getcwd()
    SAVE_DB_DIR = os.path.join(BASE_DIR, '금통위')

    if not os.path.exists(SAVE_DB_DIR):
        os.makedirs(SAVE_DB_DIR)

    #텍스트 파일 만들기
    temp_name = f"{title}.xlsx"
    txt_name = os.path.join(SAVE_DB_DIR, temp_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')

    df.to_excel(excel_writer)

    excel_writer.save()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


AttributeError: 'NoneType' object has no attribute 'find'

# 실패한거

In [ ]:
import os
import pandas as pd

for i in range(1, 45):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.content)
    li_list = []
    li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
#     print(len(li_list))
#     print(li_list)
    print(i)
    for x in range(0, len(li_list)):

        try:
            new_dict = {}
            link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
            for link in link_li:
                if link.find('a').attrs['title'][-3:] == 'pdf':
                    link_u = link.find('a').attrs['href']
                    title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                    url2 = 'http://www.bok.or.kr' + link_u
                    file_res2 = requests.get(url2)
                    file_name = '{}.pdf'.format(title)
                    with open(file_name, 'wb') as f:
                        f.write(file_res2.content)
                    try:
                        text = getTextPDF(file_name)
                        new_dict = {
                            'title' : title,
                            'text' : text
                        }
                        df = pd.DataFrame(new_dict)
                        BASE_DIR = os.getcwd()
                        SAVE_DB_DIR = os.path.join(BASE_DIR, '금통위')

                        if not os.path.exists(SAVE_DB_DIR):
                            os.makedirs(SAVE_DB_DIR)

                        #텍스트 파일 만들기
                        temp_name = f"{title}.xlsx"
                        txt_name = os.path.join(SAVE_DB_DIR, temp_name)
                        excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
                
                        df.to_excel(excel_writer)
                        
                        excel_writer.save()
                    except:
                        print('빠진 목록 : {}'.format(title))
        except:
            link2 = li_list[x].find('div').find('div').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url3 = 'http://www.bok.or.kr' + link2
            file_res3 = requests.get(url3)
            
            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res3.content)

In [ ]:
convert_pdf_to_txt(file):

In [ ]:
b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
params = {
'pageIndex' : 1
}
resp = requests.get(b_url, params = params)
soup = BeautifulSoup(resp.content)
li_list = []
li= soup.find('div', class_='bdLine type2').find('ul').find('li')

In [ ]:
li.find('div', class_='fileGoupBox').find('ul').find_all('li')[0].find('a').attrs['title'][-3:]

In [ ]:
from urllib import request
import shutil

In [ ]:
c_url = 'http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000012003&fileSn=1'
a = urlopen(c_url)

In [ ]:
resp_c = requests.get(c_url)

In [ ]:
soup = BeautifulSoup(resp_c.content)

In [ ]:
a = soup.find('div', class_='fileGoupBox')
a

In [ ]:
link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')

In [ ]:
with open('34.pdf', 'wb') as f:
    resp_c.raw.decode_content = True
    shutil.copyfileobj(resp_c.raw, f)

In [ ]:
import unicode

In [ ]:
data = request.urlopen(c_url).read()
# date2unicode(euc_bytes,'euc-kr').encode('utf-8')

text = data.decode('euc-kr')

text

In [ ]:
a

In [ ]:
 convert_pdf_to_txt(a)

In [ ]:
b = request.urlretrieve(c_url)
b

In [ ]:
 convert_pdf_to_txt(b)

In [ ]:
help(resp.raw)

In [ ]:
b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
params = {
'pageIndex' : 15
}
resp = requests.get(b_url, params = params)
soup = BeautifulSoup(resp.content)
li_list = []
li= soup.find('div', class_='bdLine type2').find('ul').find('li')
li_li = li.find_next_siblings('li')
li_list.append(li)
li_list.extend(li_li)

#     print(len(li_list))
#     print(li_list)
result_list = []
for x in range(0, len(li_list)):
    new_dict = {}
    link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
    for link in link_li:
        if link.find('a').attrs['title'][-3:] == 'pdf':
            link_u = link.find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url2 = 'http://www.bok.or.kr' + link_u
            file_res2 = requests.get(url2)
            file_name = '{}.pdf'.format(title)
            with open(file_name, 'wb') as f:
                f.write(file_res2.content)

            text = convert_pdf_to_txt(file_name)
            new_dict = {
                'title' : title,
                'text' : text
            }
            result_list.append(new_dict)
